In [328]:
import pandas as pd


In [329]:
data_df = pd.read_csv("Data_2019-20.csv",sep=';')
attribute_df = pd.read_csv("Attributes.csv")
sensitivity_df = pd.read_excel("Sensitivity_Results.xlsx")  # columns: column_name, sensitivity_level (High/Medium/Low)
kyu_df = pd.read_excel("KYU Score.xlsx")  # columns: user_id, kyu_score (0-100)

In [330]:
sensitive_df = sensitivity_df[sensitivity_df['Sensitivity_Level'].isin(['High', 'Moderate'])]

# Extract the attribute_id values (i.e., column names in data)
sensi = []
for i in sensitivity_df['Attr_id'].tolist():
    sensi.append(str(i))

In [331]:
df = data_df
n = len(df)
min_unique_ratio = 0.9  # adjust based on your needs
max_unique_ratio = 0.90  # adjust based on your needs

records = []
qids = []

for col in df.columns:
    nunique = df[col].nunique(dropna=True)  # Number of unique values excluding NaN
    ratio = nunique / n if n else 0  # Ratio of unique values to total rows
    dtype = df[col].dtype  # Data type of the column
    
    records.append({
        'column': col,
        'dtype': str(dtype),
        'nunique': nunique,
        'ratio': round(ratio, 3)
    })
    
    if 1 < nunique < n and min_unique_ratio <= ratio <= max_unique_ratio:
        qids.append(col)

summary_df = pd.DataFrame(records, columns=['column', 'dtype', 'nunique', 'ratio'])

quasi=qids


In [332]:
def run_user_query(result_df):
    # Run query
    # result_df = ps.sqldf(user_query, locals())
    
    # Determine result type
    shape = result_df.shape
    if shape[0] > 1 and shape[1] > 1:
        output_type = "table"
    elif shape[0] > 1 and shape[1] == 1:
        output_type = "column"
    elif shape[0] == 1 and shape[1] > 1:
        output_type = "row"
    else:
        output_type = "cell"

    return output_type

In [333]:
def select_strategy(data_level, sensitivity, user_trust):
    strategies = {
        'cell': {
            'high': {
                'low': ['cell_suppression', 'differential_privacy_column', 'full_masking'],
                'moderate': ['top_bottom_coding', 'microaggregation', 'partial_masking'],
                'high': ['noise_injection']
            },
            'moderate': {
                'low': ['microaggregation'],
                'moderate': ['generalization', 'partial_masking'],
                'high': ['no_transformation']
            },
            'low': {
                'low': ['noise_injection'],
                'moderate': ['no_transformation'],
                'high': ['no_transformation']
            }
        },
        'column': {
            'high': {
                'low': ['generalization', 'top_bottom_coding', 'full_masking'],
                'moderate': ['cell_suppression', 'partial_masking'],
                'high': ['noise_injection']
            },
            'moderate': {
                'low': ['binning'],
                'moderate': ['binning', 'partial_masking'],
                'high': ['no_transformation']
            },
            'low': {
                'low': ['generalization'],
                'moderate': ['no_transformation'],
                'high': ['no_transformation']
            }
        },
        'row': {
            'high': {
                'low': ['full_masking'],
                'moderate': ['microaggregation', 'partial_masking'],
                'high': ['microaggregation']
            },
            'moderate': {
                'low': ['generalization'],
                'moderate': ['microaggregation', 'partial_masking'],
                'high': ['no_transformation']
            },
            'low': {
                'low': ['microaggregation'],
                'moderate': ['no_transformation'],
                'high': ['no_transformation']
            }
        },
        'table': {
            'high': {
                'low': ['cell_suppression', 'differential_privacy', 'full_masking'],
                'moderate': ['microaggregation', 'partial_masking'],
                'high': ['microaggregation']
            },
            'moderate': {
                'low': ['generalization'],
                'moderate': ['microaggregation', 'partial_masking'],
                'high': ['no_transformation']
            },
            'low': {
                'low': ['generalization'],
                'moderate': ['no_transformation'],
                'high': ['no_transformation']
            }
        }
    }
    return strategies[data_level][sensitivity][user_trust]


In [406]:
def full_masking_cell(df, column):
    return df[column].apply(lambda x: '*' * len(str(x)))

def partial_masking_cell(df, column):
    def mask_half(val):
        val_str = str(val)
        half = len(val_str) // 2
        return '*' * half + val_str[half:]
    return df[column].apply(mask_half)

def cell_suppression(df, column, threshold=2):
    # Suppress values that occur fewer than `threshold` times
    value_counts = df[column].value_counts()
    return df[column].apply(lambda x: np.nan if value_counts[x] < threshold else x)

def noise_injection(df, column, epsilon=0.1):
    if df.shape[0] == 1 and df.shape[1] == 1:
        noise=0.85
        df[column] = df[column] +df[column].mean()* noise
    else:
        # Case 2: DataFrame with more than 1 row
        sensitivity = df[column].max() - df[column].min()
        sensitivity = max(sensitivity, 1e-6)  # Prevent divide by zero
        scale = sensitivity / epsilon  # Laplace scale
        noise = np.random.laplace(loc=0, scale=scale, size=len(df))
        df[column] = df[column] + noise
    return df 
    
def differential_privacy_column(df, column, epsilon=0.1):
    # Simple Laplace noise for demo
    return df[column] + np.random.laplace(loc=0, scale=1/epsilon, size=len(df))


def generalization_column(df, column, bins=3):
    # Bucket numerical values
    return pd.cut(df[column], bins=bins, labels=[f"Group {i+1}" for i in range(bins)])

def binning(df, column, bin_size):
    return (df[column] // bin_size) * bin_size

def top_bottom_coding(df, column, top=90, bottom=10):
    top_val = np.percentile(df[column], top)
    bottom_val = np.percentile(df[column], bottom)
    return df[column].clip(lower=bottom_val, upper=top_val)

def full_masking_row(df):
    print(df.apply(lambda row: ['XXXX' for _ in row], axis=1))
    return df.apply(lambda row: ['XXXX' for _ in row], axis=1)

def partial_masking_row(df):
    def partial(val):
        val_str = str(val)
        half = len(val_str) // 2
        return '*' * half + val_str[half:]
    return df.apply(lambda row: [partial(val) for val in row], axis=1)

def microaggregation_row(df, k=2):
    df_sorted = df.sort_values(by=df.columns[0])
    aggregated_df = df_sorted.copy()
    for i in range(0, len(df), k):
        group = df_sorted.iloc[i:i+k]
        mean_vals = group.mean(numeric_only=True)
        for col in mean_vals.index:
            aggregated_df.loc[group.index, col] = mean_vals[col]
    return aggregated_df


def full_masking_table(df):
    return df.apply(lambda row: ['XXXX' for _ in row], axis=1)


def partial_masking_table(df):
    def partial(val):
        val_str = str(val)
        half = len(val_str) // 2
        return '*' * half + val_str[half:]
    return df.applymap(partial)

def microaggregation_table(df, k=2):
    df_copy = df.copy()
    for col in df_copy.select_dtypes(include=[np.number]).columns:
        df_copy[col] = microaggregation_row(df_copy[[col]], k)[col]
    return df_copy

def differential_privacy(df, epsilon=0.1):
    df_copy = df.copy()
    for col in df_copy.select_dtypes(include=[np.number]).columns:
        df_copy[col] = df_copy[col] + np.random.laplace(0, 1/epsilon, size=len(df))
    return df_copy


In [335]:
def anonymize_by_sensitivity(df, strategies, user_trust, granularity):
    df_copy = df
    applied_strategy = None

    for strategy in strategies:
        try:
            temp_df = df_copy
            if granularity in ['cell', 'column']:
                # Attempt to apply the strategy to each column
                for column in temp_df.columns:
                    if strategy == 'cell_suppression':
                        temp_df[column] = cell_suppression(temp_df, column)
                    elif strategy == 'differential_privacy_column':
                        temp_df[column] = differential_privacy_column(temp_df, column)
                    elif strategy == 'top_bottom_coding':
                        temp_df[column] = top_bottom_coding(temp_df, column)
                    elif strategy == 'microaggregation':
                        temp_df[column] = microaggregation_row(temp_df[[column]])[column]
                    elif strategy == 'noise_injection':
                        temp_df[column] = noise_injection(temp_df, column)
                    elif strategy == 'generalization':
                        temp_df[column] = generalization_column(temp_df, column)
                    elif strategy == 'binning':
                        temp_df[column] = binning(temp_df, column, bin_size=10)
                    elif strategy == 'full_masking':
                        temp_df[column] = full_masking_cell(temp_df, column)
                    elif strategy == 'partial_masking':
                        temp_df[column] = partial_masking_cell(temp_df, column)
                    elif strategy == 'no_transformation':
                        continue
                    else:
                        raise ValueError(f"Unknown strategy: {strategy}")

            elif granularity == 'row':
                if strategy == 'full_masking':
                    temp_df.iloc[:, :] = full_masking_row(temp_df).values
                elif strategy == 'partial_masking':
                    temp_df.iloc[:, :] = partial_masking_row(temp_df).values
                elif strategy == 'microaggregation':
                    temp_df = microaggregation_row(temp_df)
                elif strategy == 'no_transformation':
                    continue
                else:
                    raise ValueError(f"Unknown strategy: {strategy}")

            elif granularity == 'table':
                if strategy == 'full_masking':
                    temp_df = full_masking_table(temp_df)
                elif strategy == 'partial_masking':
                    temp_df = partial_masking_table(temp_df)
                elif strategy == 'microaggregation':
                    temp_df = microaggregation_table(temp_df)
                elif strategy == 'differential_privacy':
                    temp_df = differential_privacy(temp_df)
                elif strategy == 'no_transformation':
                    continue
                else:
                    raise ValueError(f"Unknown strategy: {strategy}")

            # If no exception occurred, commit changes
            df_copy = temp_df
            applied_strategy = strategy
            break

        except Exception as e:
            continue  # Try next strategy if this one fails

    return df_copy, applied_strategy


In [336]:
def get_max_sensitivity_level(df, sensitivity_df):
    # Define ranking
    sensitivity_rank = {'Low': 1, 'Moderate': 2, 'High': 3}
    reverse_rank = {v: k for k, v in sensitivity_rank.items()}
    cols=list(df.columns)
    cs = [int(item) for item in cols]
    # Filter sensitivity_df to only relevant columns
    matched = sensitivity_df[sensitivity_df['Attr_id'].isin(cs)]
    # print(matched)
    if matched.empty:
        raise ValueError("No matching columns found between DataFrame and sensitivity mapping.")

    # # Map sensitivity levels to numeric and get max
    matched['level_rank'] = matched['Sensitivity_Level'].map(sensitivity_rank)

    # print(matched['level_rank'])
    max_level_num = matched['level_rank'].max()

    return reverse_rank[max_level_num]

In [337]:
import pandas as pd
import numpy as np

def calculate_anonymization_score(original_df, anonymized_df):
    assert original_df.shape == anonymized_df.shape, "DataFrames must be the same shape"
    
    n_rows, n_cols = original_df.shape
    total_cells = n_rows * n_cols
    total_distance = 0

    def delta(orig, anon):
        if pd.isnull(anon) or anon in ['*', '****', 'REDACTED', 'SUPPRESSED']:
            return 1.0
        if orig == anon:
            return 0.0
        # For generalized ranges like "30-40"
        if isinstance(anon, str) and '-' in anon:
            return 0.5
        # For numeric values: compute relative difference
        try:
            orig_val = float(orig)
            anon_val = float(anon)
            if orig_val == 0:
                return 1.0  # avoid division by zero
            return min(1.0, abs(orig_val - anon_val) / abs(orig_val))
        except:
            return 1.0  # fallback for completely different strings

    for col in original_df.columns:
        for orig, anon in zip(original_df[col], anonymized_df[col]):
            total_distance += delta(orig, anon)

    anonymization_score = total_distance / total_cells
    utility_retained = 1 - anonymization_score

    return anonymization_score, utility_retained


In [408]:
from sqlalchemy import create_engine
import sqlite3

user_id=2
# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
data_df.to_sql("data_df", conn, index=False, if_exists="replace")

# Run query
# query = 'SELECT "594","601","621","745","753","755","1341","27","29","124","5","9" FROM data_df'
query = 'SELECT "1455","1198","924","21","1524","333","351" FROM data_df where "2" in ("Gadag")'#moderate
# query = 'SELECT "155" FROM data_df where "2" in ("Gadag")'#low

result = pd.read_sql_query(query, conn)
result_type = run_user_query(result)
conn.close()
print("Result Type ------", result_type)
pseudo_map={}
# print(sensitivity_df.columns)
quasi_identifiers=quasi
sensitive_column=sensi
kyu_score = kyu_df.loc[kyu_df['ID'] == user_id, 'KYU_Score'].values[0]
print("KYU Score ------", kyu_score)

resulting_columns = result.columns.tolist()
if result_type == 'cell':
    result1=sensitivity_df[sensitivity_df['Attr_id'] == int(resulting_columns[0][1:-1])]
    # print(result1)
    sense=list(result1['Sensitivity_Level'])[0]
else:
    sense=get_max_sensitivity_level(result, sensitivity_df)
    
print("Sensitivity Level ------",sense)

strategies=select_strategy(result_type,sense.lower(),kyu_score.lower())

print("Strategies -----",strategies)
original_df=result.copy()
anonymized_df,ds = anonymize_by_sensitivity(result,strategies,kyu_score,result_type)
print("Applied---",ds)
composite,rs = calculate_anonymization_score(original_df, anonymized_df)

print(composite)


Result Type ------ row
KYU Score ------ Low
Sensitivity Level ------ High
Strategies ----- ['full_masking']
0    [XXXX, XXXX, XXXX, XXXX, XXXX, XXXX, XXXX]
dtype: object
Applied--- None
0.0


/var/folders/gl/ts61208d79v9vfwpxlybsqyh0000gn/T/ipykernel_68264/3884888596.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched['level_rank'] = matched['Sensitivity_Level'].map(sensitivity_rank)


In [402]:
result.head(5)

,1455,1198,924,21,1524,333,351
0,85,11,151193.6,13,8.37,698.0,548951


In [404]:
anonymized_df.head(5)

,1455,1198,924,21,1524,333,351
0,85,11,151193.6,13,8.37,698.0,548951
